In [51]:
from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput
import sagemaker
from sagemaker.estimator import Estimator
from sagemaker.experiments.run import Run, load_run
import time
from datetime import datetime
from sagemaker.inputs import TrainingInput
from sagemaker import image_uris

sagemaker.__version__

'2.219.0'

In [52]:
data_version = "v1"
model_version = "v1"
session = sagemaker.Session()
sm = session.sagemaker_client

experiment_name = "team1-index-predictor-model-training"
trial_suffix = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
trial_name = f"model-training-{model_version}-{trial_suffix}"

bucket_name = "team1-index-predictor-bucket"
input_prefix = f"data/processed/{data_version}"
output_bucket_prefix = f"models/{model_version}"

processing_instance_type = "ml.t3.medium"
processing_instance_count = 1

feature_group_name = 'index-predictor-feature-group-v7'

tracking_server_arn = "arn:aws:sagemaker:eu-central-1:567821811420:mlflow-tracking-server/wildfire-mj"
experiment_name = "index-predictor-model-training"


In [53]:
with Run(
    experiment_name=experiment_name,
    run_name=trial_name,
    run_display_name=trial_name,
    sagemaker_session=session,
) as run:
    experiment_config = run.experiment_config

xgboost_container = image_uris.retrieve(
    "xgboost", 
    session.boto_region_name, 
    version="1.7-1",
)

estimator = Estimator(
    image_uri=xgboost_container,
    role=sagemaker.get_execution_role(),
    instance_count=1,
    instance_type='ml.m5.large',
    output_path=f"s3://{bucket_name}/{output_bucket_prefix}",
    sagemaker_session=sagemaker.Session(),
    entry_point='../../src/models/trainer.py',
)

estimator.set_hyperparameters(
    mode="feature_store",
    dataset_sizes_path="/opt/ml/input/data/dataset_sizes/dataset_sizes.json",
    data_version=data_version,
    target_column='close_target',
    columns_to_drop='write_time,api_invocation_time,is_deleted,datetime,type,version',
    model_output_path="/opt/ml/model",
    num_trials=10,
    feature_group_name=feature_group_name,
    bucket_name=bucket_name,
    region=session.boto_region_name,
    tracking_server_arn=tracking_server_arn,
    experiment_name=experiment_name
)

estimator.fit({'dataset_sizes': f"s3://{bucket_name}/data/processed/{data_version}/dataset_sizes.json"})

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2024-06-22-09-33-04-708


2024-06-22 09:33:05 Starting - Starting the training job...
2024-06-22 09:33:20 Starting - Preparing the instances for training...
2024-06-22 09:33:44 Downloading - Downloading input data...
2024-06-22 09:34:24 Downloading - Downloading the training image......
2024-06-22 09:35:30 Training - Training image download completed. Training in progress..[2024-06-22 09:35:35.169 ip-10-0-156-208.eu-central-1.compute.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2024-06-22 09:35:35.191 ip-10-0-156-208.eu-central-1.compute.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2024-06-22:09:35:35:INFO] Imported framework sagemaker_xgboost_container.training
[2024-06-22:09:35:35:INFO] Failed to parse hyperparameter bucket_name value team1-index-predictor-bucket to Json.
Returning the value itself
[2024-06-22:09:35:35:INFO] Failed to parse hyperparameter columns_to_drop value write_time,api_invocation_time,is_deleted,datetime,type,version to Json.
Returning

In [29]:
!aws s3 ls s3://{bucket_name}/{output_bucket_prefix}/sagemaker-xgboost-2024-06-21-20-19-27-449/output/

2024-06-21 20:24:05      31602 model.tar.gz


In [54]:
import gc
gc.collect()

24255

In [11]:
!aws s3 cp s3://{bucket_name}/{output_bucket_prefix}/sagemaker-xgboost-2024-06-21-20-19-27-449/output/model.tar.gz model.tar.gz

download: s3://team1-index-predictor-bucket/models/model.xgb/sagemaker-xgboost-2024-06-18-23-40-03-184/output/model.tar.gz to ./model.tar.gz
